# Text Analysis on ottawashooting data

In [1]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\ottawashooting.csv', encoding = 'utf-8', header = None)
df.columns = ['src_tweet_id', 'src_user_id', 'src_tweet','src_date','reply_tweet_id','reply_user_id','reply_tweet','reply_date', 'label']
df.reply_tweet = df.reply_tweet.replace(np.nan,"")

In [3]:
df.columns

Index(['src_tweet_id', 'src_user_id', 'src_tweet', 'src_date',
       'reply_tweet_id', 'reply_user_id', 'reply_tweet', 'reply_date',
       'label'],
      dtype='object')

In [5]:
df

,src_tweet_id,src_user_id,src_tweet,src_date,reply_tweet_id,reply_user_id,reply_tweet,reply_date,label
0,524922078638903296,18999969,Ottawa police are confirming a shooting at the...,Wed Oct 22 13:55:50 +0000 2014,524922392582586368,2433907530,.@dominiquehardy @CBCOttawa related to St-Jean ?,Wed Oct 22 13:57:05 +0000 2014,0
1,524922078638903296,18999969,Ottawa police are confirming a shooting at the...,Wed Oct 22 13:55:50 +0000 2014,524922589861658624,15816550,@petitmetroide @CBCOttawa Il est tôt pour une ...,Wed Oct 22 13:57:52 +0000 2014,0
2,524922078638903296,18999969,Ottawa police are confirming a shooting at the...,Wed Oct 22 13:55:50 +0000 2014,524922681595269120,1206639234,Prudence avant d'en savoir plus “@CBCOttawa: O...,Wed Oct 22 13:58:14 +0000 2014,0
3,524922078638903296,18999969,Ottawa police are confirming a shooting at the...,Wed Oct 22 13:55:50 +0000 2014,524923071090917376,347531577,@StephBerthomet Ça va être rapporté partout sa...,Wed Oct 22 13:59:47 +0000 2014,0
4,524922078638903296,18999969,Ottawa police are confirming a shooting at the...,Wed Oct 22 13:55:50 +0000 2014,524923096911052802,96146974,“@CBCOttawa: Ottawa police are confirming a sh...,Wed Oct 22 13:59:53 +0000 2014,0
...,...,...,...,...,...,...,...,...,...
11389,525072912957452289,19636948,"This photo, reported to be of suspected Ottawa...",Wed Oct 22 23:55:12 +0000 2014,525100575852343297,930938779,@TdotBrownMon @Mairsy27 that soldier had nothi...,Thu Oct 23 01:45:07 +0000 2014,1
11390,525072912957452289,19636948,"This photo, reported to be of suspected Ottawa...",Wed Oct 22 23:55:12 +0000 2014,525100743146352640,100150714,@thejangler6 @Mairsy27 exactly is not the pers...,Thu Oct 23 01:45:47 +0000 2014,1
11391,525072912957452289,19636948,"This photo, reported to be of suspected Ottawa...",Wed Oct 22 23:55:12 +0000 2014,525102255515922432,322148870,@allstaradele @CP24 what a hero! Gave his life...,Thu Oct 23 01:51:48 +0000 2014,1
11392,525072912957452289,19636948,"This photo, reported to be of suspected Ottawa...",Wed Oct 22 23:55:12 +0000 2014,525102464773931008,100150714,@Mairsy27 @thejangler6 wrd but i kno wagwanin ...,Thu Oct 23 01:52:38 +0000 2014,1


## Prepocessing
- Define function to process text at one time

In [6]:
import pickle

In [7]:
'''Convert Emojis to Words'''

with open(r'D:\論文\PHEME9\Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emojis_to_word(text):
    import re
    for emot in Emoji_Dict:
        text = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), text)
    return text

In [217]:
def textprocessing(text):
    
    text = text.lower()           #lower case
    
    if 'rt' in text.split():      #cut retweet
        rt = text.find('RT')
        text = text[0:rt]  
        
    quo = text.find('“')           #cut quoting part
    quo_end = text.find('”')
    at = text.find('@')
    if quo != -1:                  #if find “
        if quo_end != -1:             # and if there is also ”
            text = text[0: quo:] + text[quo_end+1: :]
        else:
            text = text[0: quo:]
    elif quo_end != -1:
        text = text[0:at:] + text[quo_end+1: : ]
            
   
    text = convert_emojis_to_word(text)   #convert emoji to word
        
     
    
    text = " ".join([word for word in text.split() if 'http' not in word    #remove url
                                and not word.startswith(('\'@','@'))
                                and word != 'rt'])   
                 
    import re
    def remove_punct(tweet):
        new_words = []
        for word in tweet:
            w = re.sub(r'[^\w\s]','',word) #remove everything except words and space
            w = re.sub(r'_','',w) #remove underscore as well
            new_words.append(w)

        return new_words
    text = "".join(remove_punct(text))

    from nltk.tokenize import word_tokenize   
    text = word_tokenize(text)
    
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words("english"))
    stop_words=set([a for a in list(stop_words) if re.search("n't|no|not",a) ==None])
    stop_words.add('now')
    def filterstopwords(tw):
        filter_stopwords = []
        for w in tw:
            if w not in stop_words:
                filter_stopwords.append(w)
        return filter_stopwords
    text = filterstopwords(text)
    
    
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(i) for i in text]
    
#     from nltk.stem.porter import PorterStemmer
#     stem = PorterStemmer()
#     text = [stem.stem(i) for i in text]
    
    text = " ".join(text)
    
    return text

In [223]:
print("source tweet: ", df.src_tweet.iloc[15])
print("original text: ", df.reply_tweet.iloc[1678])
print("cleaned text: ", textprocessing(df.reply_tweet.iloc[1678]))

source tweet:  Ottawa police are confirming a shooting at the War Memorial. Minutes ago. No other info. #cbcOTT #OTTnews
original text:  “@disneywords: I'm like a shooting star, I've come so far. I can't go back to where I used to be. –Jasmine (Aladdin)” @marreebear7
cleaned text:  


In [216]:
quotweet = df[df.reply_tweet.str.find('“') != -1].index
quo_tweet = df[df.reply_tweet.str.find('”') != -1].index
rtt = df[df.reply_tweet.str.find('RT') != -1].index
at =  df[df.reply_tweet.str.find('@') != -1].index

set(quo_tweet).difference(set(rtt)).difference(set(quotweet)).difference(set(at))



set()

### applied on original data (**df**)

In [ ]:
df['cleaned_src_tw'] = df.src_tweet.apply(textprocessing)
df['cleaned_reply_tw'] = df.reply_tweet.apply(textprocessing)

In [ ]:
df.head()

## Sentiment Analysis

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
def get_sentiment(tweet):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(tweet)
    if vs['compound'] >=0.05:
        sentiment_label = 'Positive'
    elif (vs['compound'] > -0.05) & (vs['compound'] < 0.05):
        sentiment_label = 'Neutral'
    elif vs['compound']<= -0.05:
        sentiment_label = 'Negative' 
    result = sentiment_label
    return result

### applied on original df

In [ ]:
df['src_sentiment']= df.cleaned_src_tw.apply(get_sentiment)
df['reply_sentiment']= df.cleaned_reply_tw.apply(get_sentiment)

In [ ]:
df.head(1)

### Source tweet data

In [ ]:
src_tw_df = df.iloc[:,[0,1,2,3,8,9,11]].copy()
src_tw_df = src_tw_df.drop_duplicates()
src_tw_df.reset_index(drop=True,inplace=True)
src_tw_df

### save data to csv 

In [ ]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\ottawashooting-df.csv',index=False)
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\ottawashooting-src.csv',index=False)

## Keywords Extraction
### WordCloud

In [ ]:
from wordcloud import WordCloud

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(docx)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()

#### Reply Tweets

In [ ]:
#nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [ ]:
reply_sen = df.cleaned_reply_tw.apply(word_tokenize)
reply_tw_list = []
for sen in reply_sen:
    for token in sen:
        reply_tw_list.append(token)
reply_tw_doc = " ".join(reply_tw_list)

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(reply_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/ottawashooting/wordcloud_reply.pdf')

In [ ]:
# positive 
posi_reply_sen = df[df.reply_sentiment=='Positive'].cleaned_reply_tw.apply(word_tokenize)
posi_reply_tw_list = []
for sen in posi_reply_sen:
    for token in sen:
        posi_reply_tw_list.append(token)
posi_reply_tw_doc = " ".join(posi_reply_tw_list)


In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(posi_reply_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/ottawashooting/wordcloud_reply_posi.pdf')

#### Source Tweet

In [ ]:
src_sen = src_tw_df.cleaned_src_tw.apply(word_tokenize)
src_tw_list = []
for sen in src_sen:
    for token in sen:
        src_tw_list.append(token)

src_tw_doc = " ".join(src_tw_list)

In [ ]:
plt.figure(figsize=(20,10))
mywordcloud = WordCloud().generate(src_tw_doc)
plt.imshow(mywordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()
mywordcloud.to_file('graph/ottawashooting/wordcloud_src.pdf')

### Common

In [ ]:
from collections import Counter
freq = FreqDist(processed_text)
freq.plot(20)

In [ ]:
def get_tokens(docx,num=30):
    word_tokens = Counter(docx)
    most_common = word_tokens.most_common(num)
    result = dict(most_common)
    return result

In [ ]:
pd.DataFrame(get_tokens(reply_tw_list).items(),columns=['word','freq']).plot(kind='bar',x='word',y='freq')
plt.title("Most frequent words among reply tweets are 'Prince','show','tonight'.")

In [ ]:
pd.DataFrame(get_tokens(src_tw_list).items(),columns=['word','freq']).plot(kind='bar',x='word',y='freq')
plt.title("Most frequent words among source tweets are 'Price', 'show', 'tonight'")


## Emotion Detection

### text2emotion package
https://snyk.io/advisor/python/text2emotion 

In [ ]:
import text2emotion as te

In [ ]:
emotion = df.cleaned_reply_tw.apply(te.get_emotion)

In [ ]:
emo = emotion.apply(lambda x: max(x,key=x.get))
df['reply_emo_te'] = emo

In [ ]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\ottawashooting-df.csv',index=False)

In [ ]:
emotion1 = src_tw_df.cleaned_src_tw.apply(te.get_emotion)
emo1 = emotion.apply(lambda x: max(x,key=x.get))
src_tw_df['src_emo_te']=emo1

In [ ]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\ottawashooting-src.csv',index=False)